In [1]:
import mitiq

from mitiq import benchmarks
import numpy as np
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq
from mitiq import zne
from mitiq.zne.inference import LinearFactory

frontend = "cirq"
circuit = benchmarks.generate_rb_circuits(
  n_qubits=1, num_cliffords=2, return_type = frontend, trials=1,
)[0]

def execute(circuit, noise_level=0.01):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    executed with depolarizing noise.
    """
    # Replace with code based on your frontend and backend.
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

linear_fac = LinearFactory(scale_factors=[1.0, 2.0, 3.0])


mitigated_result = zne.execute_with_zne(circuit, execute, factory=linear_fac)

print(linear_fac.get_scale_factors())

[1.0, 2.0, 3.0]


In [1]:
from mitiq import about
about()


Mitiq: A Python toolkit for implementing error mitigation on quantum computers
Authored by: Mitiq team, 2020 & later (https://github.com/unitaryfund/mitiq)

Mitiq Version:	0.43.0

Core Dependencies
-----------------
Cirq Version:	1.4.1
NumPy Version:	1.26.4
SciPy Version:	1.15.2

Optional Dependencies
---------------------
PyQuil Version:	4.11.0
Qiskit Version:	Not installed
Braket Version:	1.91.0

Python Version:	3.10.15
Platform Info:	Darwin (x86_64)


In [3]:
print(linear_fac.get_scale_factors())

[1.0, 2.0, 3.0]


In [4]:
frontend = "cirq"

In [5]:
from mitiq import benchmarks

circuit = benchmarks.generate_rb_circuits(
  n_qubits=1, num_cliffords=2, return_type = frontend,
)[0]

print(circuit)

0: ───Y^-0.5───I───X^-0.5───I───(Z^-0.5-X^0.5)───


In [6]:
import numpy as np
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq

def execute(circuit, noise_level=0.01):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    executed with depolarizing noise.
    """
    # Replace with code based on your frontend and backend.
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

In [7]:
# Compute the expectation value of the |0><0| observable.
noisy_value = execute(circuit)
ideal_value = execute(circuit, noise_level=0.0)
print(f"Error without mitigation: {abs(ideal_value - noisy_value) :.5f}")

Error without mitigation: 0.03246


In [8]:
from mitiq import zne

mitigated_result = zne.execute_with_zne(circuit, execute)

In [9]:
print(f"Error with mitigation (ZNE): {abs(ideal_value - mitigated_result):.{3}}")

Error with mitigation (ZNE): 0.0179


In [10]:
from mitiq.zne.scaling import fold_gates_at_random

folded = fold_gates_at_random(circuit, scale_factor=2.)
print("Folded circuit:", folded, sep="\n")

Folded circuit:
0: ───Y^-0.5───Y^0.5───Y^-0.5───I───X^-0.5───X^0.5───X^-0.5───I───(Z^-0.5-X^0.5)───


In [11]:
from mitiq.zne.inference import LinearFactory

linear_fac = LinearFactory(scale_factors=[1.0, 2.0])

In [12]:
linear_fac.get_scale_factors()

[]